In [1]:
import os
import glob
from typing import Tuple, List, Union

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

In [2]:
def get_data(_type: str) -> Tuple[torch.tensor, torch.tensor]:
    """Retrieve X, Y data from the proper directory. You can
    specify whether you want it to be pulled from /train, /dev, 
    or /test, and all the *-data.csv files will be loaded in.
    """
    
    if _type not in {'train', 'dev', 'test'}:
        msg = f"{_type} not supported. Try 'train', 'dev', or 'test'."
        raise RuntimeError(msg)
    
    df = pd.DataFrame()
    
    data_path = os.path.join('data', _type, '*-data.csv')
    for fp in glob.glob(data_path):
        # Cast to float because othewise we run into a type
        # mismatch error in PyTorch
        season_df = pd.read_csv(
            fp, index_col=[0, 1], header=[0, 1, 2], dtype='float32')
        
        df = df.append(season_df)
    
    features = df[['this', 'other']]
    import pdb;pdb.set_trace()
    # Normalize features
    features = (features - features.mean()) / features.std()
    scores = df['TEAM_PTS']
    
    n_features = len(features.columns)
    n_output = len(scores.columns)
    msg = 'Uh oh, you might be losing features!'
    assert n_features + n_output == len(df.columns), msg
    
    features = torch.from_numpy(features.values)
    scores = torch.from_numpy(scores.values)
    
    return features, scores

In [3]:
def log_two_layers(n: int) -> List[Union[nn.Linear, nn.ReLU]]:
    """Given an input with n features, construct a series
    of neural network layers that decrease logarithmically.
    """
    shift_bit_length = lambda x: 1 << (x - 1).bit_length() - 1
    
    layers = []
    while n > 4:
        power_of_two = shift_bit_length(n)
        # I belive nn.Linear uses Xavier initialization
        layers.append( nn.Linear(n, power_of_two) )
        layers.append( nn.ReLU() )
        n = power_of_two
    # Now n == 4 and we add a final regression layer
    layers.append( nn.Linear(n, 1) )
    return layers

## Main Driver Logic

In [4]:
# Roughly follows https://pytorch.org/tutorials/beginner/nn_tutorial.html#

# Model hyperparameters
num_epochs = 200
batch_size = 100
learning_rate = 0.0001

# Load data as torch.tensors
x_train, y_train = get_data('train')
x_validate, y_validate = get_data('dev')

# Define our model layers by decreasing powers of two
model = nn.Sequential( *log_two_layers(x_train.shape[1]) )
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

validate_ds = TensorDataset(x_validate, y_validate)
validate_dl = DataLoader(validate_ds, batch_size=batch_size * 2)

# L1 loss is more robust to outliers
# loss_func = F.l1_loss
loss_func = F.mse_loss

for epoch in range(num_epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    with torch.no_grad():
        validate_loss = sum(loss_func(model(xb), yb) for xb, yb in validate_dl)

    # Print epoch number and average validation loss
#     print(epoch, validate_loss / len(validate_dl))

> <ipython-input-2-10ccf0021797>(25)get_data()
     23     import pdb;pdb.set_trace()
     24     # Normalize features
---> 25     features = (features - features.mean()) / features.std()
     26     scores = df['TEAM_PTS']
     27 

ipdb> features
                              this                                   \
                               PTS                                    
                                C1         F1         F2         G1   
GAME_ID    TEAM_ID                                                    
21400056.0 1.610613e+09  33.588005  22.437395  19.448509  25.802372   
           1.610613e+09  24.775633   9.426552  15.373666  21.823883   
21400058.0 1.610613e+09  12.263474  28.907562  17.357763  27.537563   
           1.610613e+09  23.563637  23.716076  11.476182  23.303366   
21400062.0 1.610613e+09  24.255468  11.301504  26.024096  28.993288   
...                            ...        ...        ...        ...   
21601228.0 1.610613e+09  20.508329  15.43

ipdb> c
> <ipython-input-2-10ccf0021797>(25)get_data()
     23     import pdb;pdb.set_trace()
     24     # Normalize features
---> 25     features = (features - features.mean()) / features.std()
     26     scores = df['TEAM_PTS']
     27 

ipdb> c


In [10]:
x_test, y_test = get_data('test')

y_predicted = model(x_test)

In [6]:
pred

tensor([[121.5803],
        [ 93.5987],
        [103.0794],
        [103.8743],
        [110.3349],
        [102.6475],
        [100.6775],
        [116.5316],
        [101.3327],
        [ 98.5718],
        [ 91.9471],
        [101.9164],
        [119.3786],
        [109.2031],
        [ 97.6315],
        [101.7076],
        [100.9606],
        [ 99.4584],
        [101.5248],
        [108.4334],
        [120.8666],
        [100.2108],
        [107.2388],
        [ 91.5200],
        [105.6588],
        [105.0902],
        [105.1649],
        [110.9239],
        [106.3023],
        [103.3936]], grad_fn=<AddmmBackward>)